In [1]:
from keras.models import load_model
from keras.utils import plot_model

import matplotlib.pyplot as plt
import os.path
import sys
import pprint
import numpy as np

from ActionRecognitionModels.data import DataSet
from DataSetPathCall import DataSetPathCall, UCF101_PathCall

#pprint.pprint(sys.path)

ucf101 = UCF101_PathCall()
ucf101.printPathList()

DATA_PATH = ucf101.definePath("dataset/UCF101/data_file.csv")
print(DATA_PATH)

MODEL_DIR = ucf101.definePath("notebook/201907/videoActionRecognitionStudy/data/checkpoints/")
print(MODEL_DIR)


Using TensorFlow backend.


['/home/s-sato/dataset/UCF101/UCF-101_frame',
 '/home/s-sato/dataset/UCF101/UCF-101_frame/train',
 '/home/s-sato/dataset/UCF101/UCF-101_frame/test',
 '']
/home/s-sato/dataset/UCF101/data_file.csv
/home/s-sato/notebook/201907/videoActionRecognitionStudy/data/checkpoints/


In [4]:
def predictStart(model_type="mlp",model_name="",seq_length=40,class_limit=4
                 ,video_name="v_ApplyLipstick_g01_c01",save_name=""):
    
    saved_model = MODEL_DIR + model_name

    #Chose images or features and image shape based on network.
    if model_type in ['conv_3d', 'c3d', 'lrcn']:
        data_type = 'images'
        image_shape = (80, 80, 3)
    elif model_type in ['lstm', 'mlp']:
        data_type = 'features'
        image_shape = None
    else:
        raise ValueError("Invalid model. See train.py for options.")
        
    model = load_model(saved_model)
    plot_model(model, to_file=model_name+'.png')
    
    # Get the data and process it.
    if image_shape is None:
        data = DataSet(seq_length=seq_length, class_limit=class_limit,data_path=DATA_PATH)
    else:
        data = DataSet(seq_length=seq_length, image_shape=image_shape,
            class_limit=class_limit,data_path=DATA_PATH)

    # Extract the sample from the data.
    sample = data.get_frames_by_filename(video_name, data_type, ucf101.getFrameDir())

    # Predict!
    prediction = model.predict(np.expand_dims(sample, axis=0))
    
    print(model_type + " accuracy")
    #print(prediction)
    data.print_class_from_prediction(np.squeeze(prediction, axis=0))
    print("")


In [5]:
predictStart(model_type="mlp",model_name="mlp-features.013-1.032.hdf5")

mlp accuracy
ApplyLipstick: 0.06
ApplyEyeMakeup: 0.00
Archery: 0.00
BabyCrawling: 0.00



In [6]:
predictStart(model_type="lstm",model_name="lstm-features.034-1.000.hdf5")

lstm accuracy
ApplyLipstick: 0.27
ApplyEyeMakeup: 0.02
Archery: 0.00
BabyCrawling: 0.00



In [7]:
predictStart(model_type="conv_3d",model_name="conv_3d-images.013-3.064.hdf5")

conv_3d accuracy
ApplyEyeMakeup: 0.42
ApplyLipstick: 0.28
Archery: 0.00
BabyCrawling: 0.00



In [8]:
predictStart(model_type="c3d",model_name="c3d-images.006-3.192.hdf5")

c3d accuracy
ApplyLipstick: 0.54
ApplyEyeMakeup: 0.21
Archery: 0.00
BabyCrawling: 0.00

